In [ ]:
!pip install faiss-cpu sentence_transformers

In [ ]:
import os
import sys
import pickle

# Detect if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    base_dir = "/content/drive/MyDrive/Smiles Discourse Analysis"
else:
    base_dir = "path/to/your/local/project/folder" # add directory if running locally

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define save directory
pickle_dir = os.path.join(base_dir, "pickles")
os.makedirs(pickle_dir, exist_ok=True)

# open  pickles correctly
with open(os.path.join(pickle_dir, 'self_help.pkl'), 'rb') as f:
    self_help_sentences = pickle.load(f)

with open(os.path.join(pickle_dir, 'thrift.pkl'), 'rb') as f:
    thrift_sentences = pickle.load(f)


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")
sh_embeddings = model.encode(self_help_sentences, convert_to_tensor=True, show_progress_bar=True)
th_embeddings = model.encode(thrift_sentences, convert_to_tensor=True, show_progress_bar=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/141 [00:00<?, ?it/s]

Batches:   0%|          | 0/186 [00:00<?, ?it/s]

In [ ]:
with open(os.path.join(pickle_dir, 'self_help_embeddings.pkl'), 'wb') as f:
    pickle.dump(sh_embeddings, f)

with open(os.path.join(pickle_dir, 'thrift_embeddings.pkl'), 'wb') as f:
    pickle.dump(th_embeddings, f)

In [ ]:
import numpy as np
from sentence_transformers import util
all_embeddings = np.vstack([sh_embeddings, th_embeddings])
similarity_matrix = util.cos_sim(sh_embeddings, th_embeddings)

In [ ]:
import numpy as np
from sentence_transformers import util
# Convert to NumPy array if needed
if not isinstance(similarity_matrix, np.ndarray):
    similarity_matrix = similarity_matrix.cpu().numpy()

# Find all indices where similarity >= threshold
threshold = 0.6
i_indices, j_indices = np.where(similarity_matrix >= threshold)
scores = similarity_matrix[i_indices, j_indices]

# Zip results into a list of (score, i, j)
matches = list(zip(scores, i_indices, j_indices))

# Sort matches so that the first results are the most similar
matches = sorted(matches, key=lambda x: x[0], reverse=True)

IndexError: list index out of range

In [ ]:
import re
# This allows you to search the matched pairs of sentences for a term
searcher = input("enter a search term: ")
pattern = r"\b"+ searcher + r"\b"

filtered_matches = [
    (score, i, j)
    for score, i, j in matches
    if re.search(pattern, self_help_sentences[i].lower()) and re.search(pattern, thrift_sentences[j].lower())
]

# Get top 5, this could be changed at convenience
top_5_matches = filtered_matches[:5]

# Display
for score, i, j in top_5_matches:
    print(f"{score:.2f}")
    print(f"UC: {self_help_sentences[i]}")
    print(f"BS: {thrift_sentences[j]}\n")

import pandas as pd

# Build DataFrame for analysis or export
df_matches = pd.DataFrame({
    "Similarity": [score for score, i, j in top_5_matches],
    "Self Help Sentence": [self_help_sentences[i] for _, i, _ in top_5_matches],
    "Thrift Sentence": [thrift_sentences[j] for _, _, j in top_5_matches]
})

print(df_matches)

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-mpnet-base-v2")

query = input("enter query here: ")
query_emb = model.encode(query, convert_to_tensor=True)

cos_scores = util.cos_sim(query_emb, all_embeddings)[0]
top_indices = [i for i, score in enumerate(cos_scores) if score > 0.6]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

# Create metadata for Boot & Shoe
sh_metadata = pd.DataFrame({
    "sentence": self_help_sentences,
    "corpus": "Boot & Shoe",
    "index": range(len(self_help_sentences))
})

# Create metadata for Unite Community
th_metadata = pd.DataFrame({
    "sentence": thrift_sentences,
    "corpus": "Unite Community",
    "index": range(len(thrift_sentences))
})

# Add offset to uc index for combined operations (optional)
th_metadata["index"] += len(self_help_sentences)

# Combine
combined_metadata = pd.concat([sh_metadata, th_metadata], ignore_index=True)


In [ ]:
import numpy as np
# stack embeddings if not already done
all_embeddings = np.vstack([sh_embeddings, th_embeddings])

import faiss

faiss.normalize_L2(all_embeddings)
dimension = all_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # inner product index

index.add(all_embeddings)  # add all vectors at once


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 15.6 MB/s eta 0:00:00


In [ ]:
query = input("Enter a query: ")
query_vec = model.encode(query, convert_to_numpy=True)
faiss.normalize_L2(query_vec.reshape(1, -1))

# Search
top_k = 4000
scores, indices = index.search(query_vec.reshape(1, -1), top_k)
results = combined_metadata.iloc[indices[0]].copy()
results["similarity"] = scores[0]


In [ ]:
# Filter sentences to include only those where the base of the query term appears.
# Using query[:-1] as a crude stemmer to capture variations like 'community', 'communities'.
# If better precision is needed, consider using spaCy or NLTK lemmatisation — though more time-consuming.
sentences = results['sentence'].to_list()
filtered_sentences = [sentence for sentence in sentences if query[:-1] in sentence]
print(len(filtered_sentences))

411


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
# You can change n_number to modify the ngram range, 2 gives bigrams, 3 trigrams.
# If you want, you can include (2,3) to have bigrams and trigrams, but that would reduce the percentages.
n_number = 2
vectorizer = CountVectorizer(ngram_range=(n_number, n_number), stop_words="english")
X = vectorizer.fit_transform(sentences)

# Sum up counts of each n-gram
sum_words = X.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

# Convert to DataFrame for inspection or export
ngram_df = pd.DataFrame(words_freq, columns=["ngram", "count"])
# These ngrams may be useful, but will not necessarily contain the query
print(ngram_df.head(20))

                  ngram  count
0        climate change    105
1           cost living     93
2           trade union     70
3        climate action     66
4       climate justice     57
5   climate environment     52
6       unite community     46
7      campaign climate     43
8   coordinators report     41
9            items food     38
10   average percentage     35
11  percentage increase     35
12           july items     35
13            food rent     35
14        rent clothing     35
15        increase july     34
16     january february     32
17    interim agreement     31
18            main body     30
19          body report     30


In [ ]:
# filter the ngrams for the query
ngram_filtered = ngram_df[ngram_df['ngram'].str.contains(query)].copy()

# Total count for those filtered ngrams
total = ngram_filtered['count'].sum()

# Add a rounded percentage column
ngram_filtered['percentage'] = (ngram_filtered['count'] / total * 100).round(2)

# Preview result
print(ngram_filtered.head(20))


                   ngram  count  percentage
0         climate change    105       11.67
3         climate action     66        7.33
4        climate justice     57        6.33
5    climate environment     52        5.78
7       campaign climate     43        4.78
45        report climate     23        2.56
72         union climate     19        2.11
105    climate emergency     12        1.33
114    climate activists     11        1.22
128   grassroots climate     10        1.11
145    climate breakdown      9        1.00
148         climate jobs      9        1.00
152      impacts climate      9        1.00
175       action climate      8        0.89
199       climate crisis      7        0.78
205         year climate      7        0.78
207        youth climate      7        0.78
244    community climate      6        0.67
246     national climate      6        0.67
256      climate strikes      6        0.67


In [ ]:
save_path = r'/content/drive/MyDrive/final exports'
os.makedirs(save_path, exist_ok=True)
ngram_df.to_csv(save_path + r'/' + query + '_ngrams.csv', index=False)